<a href="https://colab.research.google.com/github/Nikav7/TTS/blob/TTS_training/first_training_coqui_tts_vv18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train your first 🐸 TTS model 💫

### 👋 Hello and welcome to Coqui (🐸) TTS

The goal of this notebook is to show you a **typical workflow** for **training** and **testing** a TTS model with 🐸.

Let's train a very small model on a very small amount of data so we can iterate quickly.

In this notebook, we will:

1. Download data and format it for 🐸 TTS.
2. Configure the training and testing runs.
3. Train a new model.
4. Test the model and display its performance.

So, let's jump right in!


In [ ]:
## Install Coqui TTS
! pip install -U pip
! pip install TTS

## ✅ Data Preparation

### **First things first**: we need some data.

We're training a Text-to-Speech model, so we need some _text_ and we need some _speech_. Specificially, we want _transcribed speech_. The speech must be divided into audio clips and each clip needs transcription. More details about data requirements such as recording characteristics, background noise abd vocabulary coverage can be found in the [🐸TTS documentation](https://tts.readthedocs.io/en/latest/formatting_your_dataset.html).

If you have a single audio file and you need to **split** it into clips. It is also important to use a lossless audio file format to prevent compression artifacts. We recommend using **wav** file format.

The data format we will be adopting for this tutorial is taken from the widely-used  **LJSpeech** dataset, where **waves** are collected under a folder:

<span style="color:purple;font-size:15px">
/wavs<br /> 
 &emsp;| - audio1.wav<br /> 
 &emsp;| - audio2.wav<br /> 
 &emsp;| - audio3.wav<br /> 
  ...<br /> 
</span>

and a **metadata.csv** file will have the audio file name in parallel to the transcript, delimited by `|`: 
 
<span style="color:purple;font-size:15px">
# metadata.csv <br /> 
audio1|This is my sentence. <br /> 
audio2|This is maybe my sentence. <br /> 
audio3|This is certainly my sentence. <br /> 
audio4|Let this be your sentence. <br /> 
...
</span>

In the end, we should have the following **folder structure**:

<span style="color:purple;font-size:15px">
/MyTTSDataset <br /> 
&emsp;| <br /> 
&emsp;| -> metadata.csv<br /> 
&emsp;| -> /wavs<br /> 
&emsp;&emsp;| -> audio1.wav<br /> 
&emsp;&emsp;| -> audio2.wav<br /> 
&emsp;&emsp;| ...<br /> 
</span>

🐸TTS already provides tooling for the _LJSpeech_. if you use the same format, you can start training your models right away. <br /> 

After you collect and format your dataset, you need to check two things. Whether you need a **_formatter_** and a **_text_cleaner_**. <br /> The **_formatter_** loads the text file (created above) as a list and the **_text_cleaner_** performs a sequence of text normalization operations that converts the raw text into the spoken representation (e.g. converting numbers to text, acronyms, and symbols to the spoken format).

If you use a different dataset format then the LJSpeech or the other public datasets that 🐸TTS supports, then you need to write your own **_formatter_** and  **_text_cleaner_**.

## ⏳️ Loading your dataset
Load one of the dataset supported by 🐸TTS.

We will start by defining dataset config and setting LJSpeech as our target dataset and define its path.


In [2]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
import os

# BaseDatasetConfig: defines name, formatter and path of the dataset.
from TTS.tts.configs.shared_configs import BaseDatasetConfig

output_path = "/content/drive/MyDrive/LJSpeech001/output2"
if not os.path.exists(output_path):
    os.makedirs(output_path)
    

In [4]:
#!pwd

In [ ]:
#Download the REDUCED LJSpeech dataset. (just first speaker 001 - 186 files)
!wget -O $output_path/LJSpeech001.tar.bz2 https://drive.google.com/drive/folders/1UYFVOaKB8jL40qlgs_3h5fBGg4VW7ezF?usp=sharing

#!tar -xf $output_path/LJSpeech001.tar.bz2 -C $output_path

#with .tar.bz2

#DOWNLOAD LJSPEECH DATASET TO TRAIN GLOW-TTS MODEL ON IT
#!wget http://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
!wget https://drive.google.com/file/d/1ssehGMDDgNlaEWHmnEsBD0IzXSnTkrL6/view?usp=share_link/LJSpeech001.tar.bz2

#DECOMPRESS
!tar -xjf LJSpeech001.tar.bz2

In [6]:
#CONFIGURE DATASET
dataset_config = BaseDatasetConfig(
    formatter="ljspeech", meta_file_train="metadatas.csv", path=os.path.join(output_path, "/content/drive/MyDrive/LJSpeech001/")
)

## ✅ Train a new model

Let's kick off a training run 🚀🚀🚀.

Deciding on the model architecture you'd want to use is based on your needs and available resources. Each model architecture has it's pros and cons that define the run-time efficiency and the voice quality.
We have many recipes under `TTS/recipes/` that provide a good starting point. For this tutorial, we will be using `GlowTTS`.

We will begin by initializing the model training configuration.

In [10]:
# GlowTTSConfig: all model related values for training, validating and testing.
from TTS.tts.configs.glow_tts_config import GlowTTSConfig
config = GlowTTSConfig(
    batch_size=32,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=3,
    text_cleaner="phoneme_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    print_step=25,
    print_eval=False,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    save_step=1000,
)

In [ ]:
#install cython to prevent errors in the inizialization of the audio processor
!pip install -U Cython

Next we will initialize the audio processor which is used for feature extraction and audio I/O.

In [ ]:
from TTS.utils.audio import AudioProcessor
ap = AudioProcessor.init_from_config(config)

Next we will initialize the tokenizer which is used to convert text to sequences of token IDs.  If characters are not defined in the config, default characters are passed to the config.

In [12]:
from TTS.tts.utils.text.tokenizer import TTSTokenizer
tokenizer, config = TTSTokenizer.init_from_config(config)

Next we will load data samples. Each sample is a list of ```[text, audio_file_path, speaker_name]```. You can define your custom sample loader returning the list of samples.

In [13]:
from TTS.tts.datasets import load_tts_samples
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

 | > Found 186 files in /content/drive/MyDrive/LJSpeech001


Now we're ready to initialize the model.

Models take a config object and a speaker manager as input. Config defines the details of the model like the number of layers, the size of the embedding, etc. Speaker manager is used by multi-speaker models.

In [14]:
from TTS.tts.models.glow_tts import GlowTTS
model = GlowTTS(config, ap, tokenizer, speaker_manager=None)

Trainer provides a generic API to train all the 🐸TTS models with all its perks like mixed-precision training, distributed training, etc.

In [15]:
from trainer import Trainer, TrainerArgs
trainer = Trainer(
    TrainerArgs(), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

 > Training Environment:
 | > Num. of CPUs: 2
 | > Num. of Torch Threads: 1
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False

 > Model has 28610257 parameters


### AND... 3,2,1... START TRAINING 🚀🚀🚀

In [18]:
trainer.fit()


 > EPOCH: 0/3
 --> /content/drive/MyDrive/LJSpeech001/output2/run-November-24-2022_12+53PM-0000000


[*] Pre-computing phonemes...


  1%|          | 1/185 [00:00<00:33,  5.56it/s]

ðɛɹ taɪp ɪz ɔn ðə laɪnz əv ðə d͡ʒɚmən ænd fɹɛnt͡ʃ ɹæðɚ ðən əv ðə ɹoʊmən pɹɪntɚz.
 [!] Character '͡' not found in the vocabulary. Discarding it.


100%|██████████| 185/185 [00:16<00:00, 11.04it/s]


...


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.15060 (-0.17771)
     | > avg_loss: 3.37966 (-0.00395)
     | > avg_log_mle: 0.75288 (-0.00335)
     | > avg_loss_dur: 2.62678 (-0.00060)

 > BEST MODEL : /content/drive/MyDrive/LJSpeech001/output2/run-November-24-2022_12+53PM-0000000/best_model_12.pth

 > EPOCH: 2/3
 --> /content/drive/MyDrive/LJSpeech001/output2/run-November-24-2022_12+53PM-0000000

 > TRAINING (2022-11-24 13:02:09) 




> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: en-us
		| > phoneme backend: gruut
	| > 1 not found characters:
	| > ͡
| > Number of instances : 185
 | > Preprocessing samples
 | > Max text length: 169
 | > Min text length: 26
 | > Avg text length: 98.91891891891892
 | 
 | > Max audio length: 220083.0
 | > Min audio length: 39347.0
 | > Avg audio length: 142243.77837837837
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.



 > EVALUATION 





> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: en-us
		| > phoneme backend: gruut
	| > 1 not found characters:
	| > ͡
| > Number of instances : 1
 | > Preprocessing samples
 | > Max text length: 62
 | > Min text length: 62
 | > Avg text length: 62.0
 | 
 | > Max audio length: 81075.0
 | > Min audio length: 81075.0
 | > Avg audio length: 81075.0
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.
 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.20784 (+0.05725)
     | > avg_loss: 3.37928 (-0.00038)
     | > avg_log_mle: 0.75287 (-0.00001)
     | > avg_loss_dur: 2.62641 (-0.00038)

 > BEST MODEL : /content/drive/MyDrive/LJSpeech001/output2/run-November-24-2022_12+53PM-0000000/best_model_18.pth


#### 🚀 Run the Tensorboard. 🚀
On the notebook and Tensorboard, you can monitor the progress of your model. Also Tensorboard provides certain figures and sample outputs.

In [ ]:
!pip install tensorboard
!tensorboard --logdir=/content/drive/MyDrive/LJSpeech001/output2

## ✅ Test the model

We made it! 🙌

Let's kick off the testing run, which displays performance metrics.

We're committing the cardinal sin of ML 😈 (aka - testing on our training data) so you don't want to deploy this model into production. In this notebook we're focusing on the workflow itself, so it's forgivable 😇

You can see from the test output that our tiny model has overfit to the data, and basically memorized this one sentence.

When you start training your own models, make sure your testing data doesn't include your training data 😅

Let's get the latest saved checkpoint. 

In [25]:
import glob, os
output_path = "/content/drive/MyDrive/LJSpeech001/output2"
ckpts = sorted([f for f in glob.glob(output_path+"/*/*.pth")])
configs = sorted([f for f in glob.glob(output_path+"/*/*.json")])

In [26]:
!tts --text "Text for TTS" \
      --model_path $ckpts \
      --config_path $configs \
      --out_path out.wav

# !tts --text "the sound is horrible" \
#     --model_path /content/drive/MyDrive/LJSpeech001/output/ckpts \
#     --config_path /content/drive/MyDrive/LJSpeech001/output/configs \
#     --out_path /content/out.wav

# !tts --text "the sound is horrible" \
#     --model_path /content/drive/MyDrive/trainoutput/run/best_model.pth \
#     --config_path /content/drive/MyDrive/trainoutput/run/config.json \
#     --out_path /content/output.wav

usage: tts [-h] [--list_models [LIST_MODELS]]
           [--model_info_by_idx MODEL_INFO_BY_IDX]
           [--model_info_by_name MODEL_INFO_BY_NAME] [--text TEXT]
           [--model_name MODEL_NAME] [--vocoder_name VOCODER_NAME]
           [--config_path CONFIG_PATH] [--model_path MODEL_PATH]
           [--out_path OUT_PATH] [--use_cuda USE_CUDA]
           [--vocoder_path VOCODER_PATH]
           [--vocoder_config_path VOCODER_CONFIG_PATH]
           [--encoder_path ENCODER_PATH]
           [--encoder_config_path ENCODER_CONFIG_PATH]
           [--speakers_file_path SPEAKERS_FILE_PATH]
           [--language_ids_file_path LANGUAGE_IDS_FILE_PATH]
           [--speaker_idx SPEAKER_IDX] [--language_idx LANGUAGE_IDX]
           [--speaker_wav SPEAKER_WAV [SPEAKER_WAV ...]]
           [--gst_style GST_STYLE]
           [--capacitron_style_wav CAPACITRON_STYLE_WAV]
           [--capacitron_style_text CAPACITRON_STYLE_TEXT]
           [--list_speaker_idxs [LIST_SPEAKER_IDXS]]
           [-

## 📣 Listen to the synthesized wave 📣

In [ ]:
import IPython
IPython.display.Audio("out.wav")

## 🎉 Congratulations! 🎉 You now have trained your first TTS model! 
Follow up with the next tutorials to learn more advanced material.